In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv("fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data.shape

(20800, 5)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
data = data.drop(['id'],axis=1)

In [13]:
data = data.fillna('')

In [15]:
data["content"]=data["title"]+" "+data["author"]+" "+data["text"]

In [17]:
data = data.drop(['title','author','text'],axis=1)

In [19]:
data.head()

,label,content
0,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,1,Why the Truth Might Get You Fired Consortiumne...
3,1,15 Civilians Killed In Single US Airstrike Hav...
4,1,Iranian woman jailed for fictional unpublished...


In [21]:
data['content'] = data['content'].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [22]:
data['content'] = data['content'].str.replace('[^\w\s]','') 


<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\bknai\AppData\Local\Temp\ipykernel_18712\1705779514.py:1: SyntaxWarning: invalid escape sequence '\w'
  data['content'] = data['content'].str.replace('[^\w\s]','')


In [25]:
import nltk 
nltk.download('stopwords') 
# Remove stop words 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bknai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
from nltk.corpus import stopwords 
stop = stopwords.words('english') 
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) 


In [29]:
from nltk.corpus import stopwords 
stop = stopwords.words('english') 
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))  

In [31]:
from nltk.stem import WordNetLemmatizer 
from textblob import Word 
data['content'] = data['content'].apply(lambda x: " ".join([Word (word).lemmatize() for word in x.split()])) 
data['content'].head() 
X = data[['content']] 
y = data['label']

In [43]:
from sklearn.model_selection import train_test_split

In [45]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=45, stratify=y) 
print (X_train.shape) 
print (y_train.shape) 
print (X_test.shape) 
print (y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000) 
tfidf_vect.fit(data['content']) 
xtrain_tfidf = tfidf_vect.transform(X_train['content']) 
xtest_tfidf = tfidf_vect.transform(X_test['content'])

In [55]:
from sklearn.linear_model import PassiveAggressiveClassifier 
from sklearn import metrics 
pclf = PassiveAggressiveClassifier() 
pclf.fit(xtrain_tfidf, y_train) 
predictions = pclf.predict(xtest_tfidf) 
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96      3116
           1       0.96      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [57]:
print(metrics.confusion_matrix(y_test,predictions))

[[2998  118]
 [ 123 3001]]


In [59]:
from sklearn.neural_network import MLPClassifier 
mlpclf = MLPClassifier(hidden_layer_sizes=(256,64,16), activation = 'relu', solver = 'adam') 
mlpclf.fit(xtrain_tfidf, y_train) 
predictions = mlpclf.predict(xtest_tfidf) 
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      3116
           1       0.97      0.96      0.96      3124

    accuracy                           0.96      6240
   macro avg       0.96      0.96      0.96      6240
weighted avg       0.96      0.96      0.96      6240



In [61]:
print(metrics.confusion_matrix(y_test,predictions))

[[3009  107]
 [ 121 3003]]


In [73]:
import pickle
pickle.dump(mlpclf, open("fakenews1.pkl", "wb"))

In [ ]:
loaded_model = pickle.load(open("fakenews1.pkl", 'rb'))

result = loaded_model.score(X_test.values, y_test.values)

print(result)